In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# 1.⁠ ⁠mortgage: calculated

# 2.⁠ ⁠rentals: ZORI(smoothed): all homes plus multifamily time series - metro&us
# The index is dollar-denominated by computing the mean of listed rents that fall 
# into the 40th to 60th percentile range for all homes and apartments in a given
# region, which is weighted to reflect the rental housing stock.
rentals = pd.read_csv("data/Dataset to use/Metro_zori_uc_sfrcondomfr_sm_month.csv")
# print(rentals.head())

# 3.⁠ ⁠SALES: median sale price (smooth, all homes, monthly) - metro&us
# The Sales Count Nowcast is the estimated number of unique properties that sold 
# during the month after accounting for the latency between when sales occur and 
# when they are reported. Available only for the raw cut of all homes.
# Sale Price (median/mean): The price at which homes across various geographies were sold.
# Sale-to-List Ratio (mean/median): Ratio of sale vs. final list price.
# Percent of Sales Below/Above List: Share of sales where sale price below/above the 
# final list price; excludes homes sold for exactly the list price.
median_sales = pd.read_csv("data/Dataset to use/Metro_median_sale_price_uc_sfrcondo_sm_month.csv")
# print(median_sales.head())

# 4.⁠ ⁠ZHVF (Forecast), All homes (SFR, condo/co-op), raw, mid-tier - metro & us
# A month-ahead, quarter-ahead and year-ahead forecast of the Zillow Home Value Index (ZHVI).
home_forecast = pd.read_csv("data/Dataset to use/Metro_zhvf_growth_uc_sfrcondo_tier_0.33_0.67_month.csv")
print(home_forecast.head())


   RegionID  SizeRank       RegionName RegionType StateName    BaseDate  \
0    102001         0    United States    country       NaN  2024-02-29   
1    394913         1     New York, NY        msa        NY  2024-02-29   
2    753899         2  Los Angeles, CA        msa        CA  2024-02-29   
3    394463         3      Chicago, IL        msa        IL  2024-02-29   
4    394514         4       Dallas, TX        msa        TX  2024-02-29   

   2024-03-31  2024-05-31  2025-02-28  
0         0.8         2.2         0.6  
1         0.5         1.0        -1.6  
2         1.0         2.3        -0.9  
3         1.1         3.0        -1.0  
4         1.1         2.8         0.8  


In [3]:
# Data cleaning for rentals dataset 

# missing value check
rentals.dropna(subset=['RegionID', 'RegionName'], inplace=True)

# long format 
rentals_melted = rentals.melt(id_vars=["RegionID", "SizeRank", "RegionName", "RegionType", "StateName"],
                              var_name="Date",
                              value_name="Rent")

# Convert 'Date' to datetime format
rentals_melted['Date'] = pd.to_datetime(rentals_melted['Date'], errors='coerce')

# Drop rows where 'Rent' is null
rentals_melted.dropna(subset=['Rent'], inplace=True)


# Drop all rows where RegionType is country
state_rentals = rentals_melted[rentals_melted['RegionType'] != 'country']

state_rentals = state_rentals.copy()
state_rentals.rename(columns={
    'RegionName': 'Region Name',
    'RegionType': 'Region Type',
    'StateName': 'State Name',
    'Date': 'Date Recorded',
    'Rent': 'Monthly Rent'
}, inplace=True)

state_abbreviations = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado',
    'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana',
    'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia',
    'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}

# Replace state abbreviations with full names in the 'State Name' column
state_rentals['State Name'] = state_rentals['State Name'].map(state_abbreviations).fillna(state_rentals['State Name'])


print(state_rentals.head())

state_rentals.to_csv("Cleaned_Data/cleaned_state_rentals.csv", index=False)


   RegionID  SizeRank      Region Name Region Type  State Name Date Recorded  \
1    394913         1     New York, NY         msa    New York    2015-01-31   
2    753899         2  Los Angeles, CA         msa  California    2015-01-31   
3    394463         3      Chicago, IL         msa    Illinois    2015-01-31   
4    394514         4       Dallas, TX         msa       Texas    2015-01-31   
5    394692         5      Houston, TX         msa       Texas    2015-01-31   

   Monthly Rent  
1   2286.918320  
2   1833.212831  
3   1418.891001  
4   1104.552997  
5   1226.598136  


In [4]:
# Convert 'Date' to datetime format
median_sales_melted = median_sales.melt(id_vars=["RegionID", "SizeRank", "RegionName", "RegionType", "StateName"],
                                        var_name="Date",
                                        value_name="MedianSalePrice")
median_sales_melted['Date'] = pd.to_datetime(median_sales_melted['Date'], errors='coerce')

# Drop rows where 'MedianSalePrice' is null
median_sales_melted.dropna(subset=['MedianSalePrice'], inplace=True)

# Drop all rows where RegionType is country
state_median_sales = median_sales_melted[median_sales_melted['RegionType'] != 'country']
state_median_sales = state_median_sales.copy()

# Rename columns
state_median_sales.rename(columns={
    'RegionName': 'Region Name',
    'RegionType': 'Region Type',
    'StateName': 'State Name',
    'Date': 'Date Recorded',
    'MedianSalePrice': 'Median Sale Price'
}, inplace=True)

# Full state names
state_abbreviations = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado',
    'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana',
    'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia',
    'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}
state_median_sales['State Name'] = state_median_sales['State Name'].map(state_abbreviations).fillna(state_median_sales['State Name'])

print(state_median_sales.head())


   RegionID  SizeRank      Region Name Region Type  State Name Date Recorded  \
1    394913         1     New York, NY         msa    New York    2008-04-30   
2    753899         2  Los Angeles, CA         msa  California    2008-04-30   
3    394463         3      Chicago, IL         msa    Illinois    2008-04-30   
4    394514         4       Dallas, TX         msa       Texas    2008-04-30   
5    394692         5      Houston, TX         msa       Texas    2008-04-30   

   Median Sale Price  
1           395000.0  
2           461000.0  
3           227333.0  
4           142967.0  
5           145788.0  


In [5]:
# Long format
home_forecast_melted = home_forecast.melt(id_vars=["RegionID", "SizeRank", "RegionName", "RegionType", "StateName", "BaseDate"],
                                          var_name="ForecastDate",
                                          value_name="ForecastGrowth")

# Datetime
home_forecast_melted['BaseDate'] = pd.to_datetime(home_forecast_melted['BaseDate'], errors='coerce')
home_forecast_melted['ForecastDate'] = pd.to_datetime(home_forecast_melted['ForecastDate'], errors='coerce')

# Drop rows where 'ForecastGrowth' is null
home_forecast_melted.dropna(subset=['ForecastGrowth'], inplace=True)

# Drop all rows where RegionType is country
state_home_forecast = home_forecast_melted[home_forecast_melted['RegionType'] != 'country']
state_home_forecast = state_home_forecast.copy()

# Rename columns
state_home_forecast.rename(columns={
    'RegionName': 'Region Name',
    'RegionType': 'Region Type',
    'StateName': 'State Name',
    'BaseDate': 'Base Date Recorded',
    'ForecastDate': 'Forecast Date',
    'ForecastGrowth': 'Forecast Growth (%)'
}, inplace=True)

# Full state names
state_abbreviations = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado',
    'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana',
    'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia',
    'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}
state_home_forecast['State Name'] = state_home_forecast['State Name'].map(state_abbreviations).fillna(state_home_forecast['State Name'])

state_home_forecast.to_csv("Cleaned_Data/cleaned_state_home_forecast.csv", index=False)

print(state_home_forecast.head())


   RegionID  SizeRank      Region Name Region Type  State Name  \
1    394913         1     New York, NY         msa    New York   
2    753899         2  Los Angeles, CA         msa  California   
3    394463         3      Chicago, IL         msa    Illinois   
4    394514         4       Dallas, TX         msa       Texas   
5    394692         5      Houston, TX         msa       Texas   

  Base Date Recorded Forecast Date  Forecast Growth (%)  
1         2024-02-29    2024-03-31                  0.5  
2         2024-02-29    2024-03-31                  1.0  
3         2024-02-29    2024-03-31                  1.1  
4         2024-02-29    2024-03-31                  1.1  
5         2024-02-29    2024-03-31                  0.8  


In [6]:

# Dictionary of state names to their approximate geographic center coordinates
state_coords = {
    'Alabama': (32.806671, -86.791130),
    # 'Alaska': (61.370716, -152.404419),
    'Arizona': (33.729759, -111.431221),
    'Arkansas': (34.969704, -92.373123),
    'California': (36.116203, -119.681564),
    'Colorado': (39.059811, -105.311104),
    'Connecticut': (41.597782, -72.755371),
    'Delaware': (39.318523, -75.507141),
    'Florida': (27.766279, -81.686783),
    'Georgia': (33.040619, -83.643074),
    # 'Hawaii': (21.094318, -157.498337),
    'Idaho': (44.240459, -114.478828),
    'Illinois': (40.349457, -88.986137),
    'Indiana': (39.849426, -86.258278),
    'Iowa': (42.011539, -93.210526),
    'Kansas': (38.526600, -96.726486),
    'Kentucky': (37.668140, -84.670067),
    'Louisiana': (31.169546, -91.867805),
    'Maine': (44.693947, -69.381927),
    'Maryland': (39.063946, -76.802101),
    'Massachusetts': (42.230171, -71.530106),
    'Michigan': (43.326618, -84.536095),
    'Minnesota': (45.694454, -93.900192),
    'Mississippi': (32.741646, -89.678696),
    'Missouri': (38.456085, -92.288368),
    'Montana': (46.921925, -110.454353),
    'Nebraska': (41.125370, -98.268082),
    'Nevada': (38.313515, -117.055374),
    'New Hampshire': (43.452492, -71.563896),
    'New Jersey': (40.298904, -74.521011),
    'New Mexico': (34.840515, -106.248482),
    'New York': (42.165726, -74.948051),
    'North Carolina': (35.630066, -79.806419),
    'North Dakota': (47.528912, -99.784012),
    'Ohio': (40.388783, -82.764915),
    'Oklahoma': (35.565342, -96.928917),
    'Oregon': (44.572021, -122.070938),
    'Pennsylvania': (40.590752, -77.209755),
    'Rhode Island': (41.680893, -71.511780),
    'South Carolina': (33.856892, -80.945007),
    'South Dakota': (44.299782, -99.438828),
    'Tennessee': (35.747845, -86.692345),
    'Texas': (31.054487, -97.563461),
    'Utah': (40.150032, -111.862434),
    'Vermont': (44.045876, -72.710686),
    'Virginia': (37.769337, -78.169968),
    'Washington': (47.400902, -121.490494),
    'West Virginia': (38.491226, -80.954570),
    'Wisconsin': (44.268543, -89.616508),
    'Wyoming': (42.755966, -107.302490)
}


coords_df = pd.DataFrame(list(state_coords.items()), columns=['State Name', 'Coordinates'])

# Split the 'Coordinates' column into two separate columns for latitude and longitude
coords_df[['Latitude', 'Longitude']] = pd.DataFrame(coords_df['Coordinates'].tolist(), index=coords_df.index)

state_rentals = pd.merge(state_rentals, coords_df, on='State Name', how='left')

state_rentals.to_csv("Cleaned_Data/cleaned_state_rentals.csv", index=False)

state_median_sales = pd.merge(state_median_sales, coords_df, on='State Name', how='left')

state_median_sales.to_csv("Cleaned_Data/cleaned_median_sales.csv", index=False)

print(state_median_sales.head())

   RegionID  SizeRank      Region Name Region Type  State Name Date Recorded  \
0    394913         1     New York, NY         msa    New York    2008-04-30   
1    753899         2  Los Angeles, CA         msa  California    2008-04-30   
2    394463         3      Chicago, IL         msa    Illinois    2008-04-30   
3    394514         4       Dallas, TX         msa       Texas    2008-04-30   
4    394692         5      Houston, TX         msa       Texas    2008-04-30   

   Median Sale Price               Coordinates   Latitude   Longitude  
0           395000.0   (42.165726, -74.948051)  42.165726  -74.948051  
1           461000.0  (36.116203, -119.681564)  36.116203 -119.681564  
2           227333.0   (40.349457, -88.986137)  40.349457  -88.986137  
3           142967.0   (31.054487, -97.563461)  31.054487  -97.563461  
4           145788.0   (31.054487, -97.563461)  31.054487  -97.563461  


In [7]:
## Datasets with hawaii and alaska dropped for geographical plotting

# exclude Hawaii and Alaska for state_rentals
state_rentals_without_hawaii_alaska = state_rentals[~state_rentals['State Name'].isin(['Hawaii', 'Alaska'])]

state_rentals_without_hawaii_alaska.to_csv("Cleaned_Data/state_rentals_excluding_hawaii_alaska.csv", index=False)

median_sales_without_hawaii_alaska = state_median_sales[~state_median_sales['State Name'].isin(['Hawaii', 'Alaska'])]

median_sales_without_hawaii_alaska.to_csv("Cleaned_Data/median_sales_excluding_hawaii_alaska.csv", index=False)

print(state_rentals_without_hawaii_alaska['State Name'].unique())
print(median_sales_without_hawaii_alaska['State Name'].unique())



['New York' 'California' 'Illinois' 'Texas' 'Virginia' 'Pennsylvania'
 'Florida' 'Georgia' 'Massachusetts' 'Arizona' 'Michigan' 'Washington'
 'Minnesota' 'Colorado' 'Maryland' 'Missouri' 'North Carolina' 'Oregon'
 'Ohio' 'Nevada' 'Indiana' 'Tennessee' 'Rhode Island' 'Wisconsin'
 'Oklahoma' 'Kentucky' 'Louisiana' 'Utah' 'Connecticut' 'Alabama'
 'Nebraska' 'South Carolina' 'New Mexico' 'Idaho' 'Arkansas' 'Iowa'
 'Kansas' 'Maine' 'New Hampshire' 'Mississippi' 'Delaware' 'New Jersey'
 'Montana' 'Wyoming' 'South Dakota' 'North Dakota' 'West Virginia'
 'Vermont']
['New York' 'California' 'Illinois' 'Texas' 'Virginia' 'Pennsylvania'
 'Florida' 'Georgia' 'Massachusetts' 'Arizona' 'Michigan' 'Washington'
 'Minnesota' 'Colorado' 'Maryland' 'Missouri' 'North Carolina' 'Oregon'
 'Ohio' 'Nevada' 'Indiana' 'Tennessee' 'Rhode Island' 'Wisconsin'
 'Oklahoma' 'Kentucky' 'Louisiana' 'Utah' 'Connecticut' 'Alabama'
 'Nebraska' 'South Carolina' 'New Mexico' 'Idaho' 'Arkansas' 'Iowa'
 'Kansas' 'Mississippi'

In [8]:
## Data cleaning for future values dataset 
future = pd.read_csv('data/Dataset to use/Metro_zhvf_growth_uc_sfrcondo_tier_0.33_0.67_month.csv')
# future.head()

missing_data = future.isnull().sum()

# Convert 'BaseDate' column to datetime
future['BaseDate'] = pd.to_datetime(future['BaseDate'])

# Renaming columns
future.rename(columns={
    '2024-03-31': 'March_2024_Forecast',
    '2024-05-31': 'May_2024_Forecast',
    '2025-02-28': 'February_2025_Forecast'
}, inplace=True)

# Dropping country values
future = future[future['RegionType'] != 'country']

future.head(), missing_data

future.to_csv("Cleaned_Data/cleaned_future_sales.csv", index=False)



In [9]:
city_data = pd.read_excel('data/uscities.xlsx')

city_data = city_data.drop(columns=[col for col in city_data.columns if col not in ['city', 'lat', 'lng', 'state_name']])

# Ensure the city and state names in both datasets are ready for merging
state_median_sales['city'] = state_median_sales['Region Name'].str.extract(r'^([^,]+),')[0]  # Extract city names
state_median_sales['state_name'] = state_median_sales['State Name']  # Ensure state names are aligned

# You mentioned city_data is ready with 'city' and 'state_name' columns, so we can proceed directly

# Merge the datasets on both 'city' and 'state_name'
merged_df = pd.merge(state_median_sales, city_data, on=['city', 'state_name'], how='left')

# Check the merge result
# print(merged_df.head())

# If needed, filter by any specific conditions or dates
# For example, to filter by year 2023 as in previous contexts
merged_df['Date Recorded'] = pd.to_datetime(merged_df['Date Recorded'])
merged_df_2023 = merged_df[merged_df['Date Recorded'].dt.year == 2023]

merged_df_2023 = merged_df_2023.dropna(subset=['lat', 'lng'])

state_rentals['city'] = state_rentals['Region Name'].str.extract(r'^([^,]+),')[0]
state_rentals['state_name'] = state_rentals['State Name']

merged_rentals = pd.merge(state_rentals, city_data, on=['city', 'state_name'], how='left')
merged_rentals_cleaned = merged_rentals.dropna(subset=['lat', 'lng'])
merged_rentals_cleaned = merged_rentals_cleaned[merged_rentals_cleaned['Date Recorded'].dt.year == 2023]
# Dropping outlier where rent average is 16,000 a month in 2023
merged_rentals_cleaned = merged_rentals_cleaned[merged_rentals_cleaned['Region Name'] != 'Glenwood Springs, CO']

merged_rentals_cleaned.head()
merged_rentals_cleaned.to_csv("Cleaned_Data/2023_rental_price_city.csv")
# Display the head of the merged data to check correctness
# print(merged_df_2023.head())
# merged_df_2023.to_csv("Cleaned_Data/2023_sale_price.csv")
# print(state_rentals.head())

In [10]:
state_rentals_without_hawaii_alaska['Date Recorded'] = pd.to_datetime(state_rentals_without_hawaii_alaska['Date Recorded'])
state_rentals_without_hawaii_alaska = state_rentals_without_hawaii_alaska[state_rentals_without_hawaii_alaska['Date Recorded'].dt.year == 2023]

state_rentals_without_hawaii_alaska.to_csv("Cleaned_Data/2023_rental_price.csv", index=False)

/var/folders/nq/x683qlmx39ng8v_l8ymc6cw00000gn/T/ipykernel_65183/3313705730.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_rentals_without_hawaii_alaska['Date Recorded'] = pd.to_datetime(state_rentals_without_hawaii_alaska['Date Recorded'])


In [11]:
median_sales_without_hawaii_alaska['Date Recorded'] = pd.to_datetime(median_sales_without_hawaii_alaska['Date Recorded'])
median_sales_without_hawaii_alaska = median_sales_without_hawaii_alaska[median_sales_without_hawaii_alaska['Date Recorded'].dt.year == 2023]

median_sales_without_hawaii_alaska.to_csv("Cleaned_Data/2023_sales_state.csv", index=False)

/var/folders/nq/x683qlmx39ng8v_l8ymc6cw00000gn/T/ipykernel_65183/2576744259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  median_sales_without_hawaii_alaska['Date Recorded'] = pd.to_datetime(median_sales_without_hawaii_alaska['Date Recorded'])
